This notebook should help with the analysis of the results from the first application of the correction of values.
The research question is 

- Is there a correlation between degradation and overshooting of predictions?
- Should the restricted area be calculated using the truth@t-1 and truth@t-2 instead of truth@t-1 and pred@t-1?
- Should the restricted area be depending on the length of 1s/0s on the prediction?
- How is the degradation distributed?

- How big is the influence of the correction if the restriction is too big?
- How to calculate beta and parts of the correction?
- How to consider dimension jumps in the correction?

In [103]:
from glob import glob
import pandas as pd
from io import StringIO

Reading in the data from the log files as pandas panel objects.

In [104]:
method = "PreviousError"
# method = "Delta2PowerTwo"

In [105]:
def read_file(filename, methodshorthand):
    df = ""
    if methodshorthand == "prer":
        with open(filename, 'r') as f:
            file_str = [line.split(',', 1)[1] for line in f.readlines() if "," in line and "offset" not in line]
        file_str = StringIO("".join(file_str))
        df = pd.read_csv(file_str, index_col=0, header=None)
        df.columns = ["uncorrected", "corrected", "truth", "overshot", "offset", "empty"]
        df.index.name = "index"
    elif methodshorthand == "d2p2":
        with open(logfiles[num], 'r') as f:
            file_str = [line.split(',', 1)[1] for line in f.readlines() if "," in line and "restricted" not in line]
        file_str = StringIO("".join(file_str))
        df = pd.read_csv(file_str, index_col=0, header=None)
        df.columns = ["uncorrected", "corrected", "truth", "overshot", "restricted", "delta"]
        df.index.name = "index"
    return df

def get_panel(method):
    handles = dict(PreviousError="prer", Delta2PowerTwo="d2p2")
    shorthand = handles[method]
    logfiles =  './*{}.log'.format(shorthand)
    logfiles = glob(logfiles)
    return pd.Panel({fname:read_file(fname, shorthand) for fname in logfiles})

In [106]:
panel = get_panel(method)

/home/ucyo/Environments/infospace/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  """Entry point for launching an IPython kernel.


Actual processing of the data